In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/inputs/y_train.npy
/kaggle/input/inputs/X_train.npy
/kaggle/input/rossmann-store-sales/sample_submission.csv
/kaggle/input/rossmann-store-sales/store.csv
/kaggle/input/rossmann-store-sales/train.csv
/kaggle/input/rossmann-store-sales/test.csv


In [43]:
import torch
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
import pandas as pd
import numpy as np

In [44]:
store_df=pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv",low_memory=False)
train_df=pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv",parse_dates=['Date'],low_memory=False)
test_df=pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv",parse_dates=['Date'],low_memory=False)
train_df


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [45]:
# merging stores with train and test data
train=pd.merge(train_df,store_df,on='Store',how='inner')
test=pd.merge(test_df,store_df,on='Store',how='inner')

train


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  object        
 8   SchoolHoliday              1017209 non-null  int64         
 9   StoreType                  1017209 non-null  object        
 10  Assortment                 1017209 non-null  object        
 11  CompetitionDistance        1014567 no

In [47]:
# 1. dealing with missing values
train.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64

In [48]:
train=train.sort_values(['Store', 'Date'])
test=test.sort_values(['Store', 'Date'])
test_sorted = test.sort_values(by=['Store', 'Date']).reset_index(drop=True)
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
1016095,1,2,2013-01-01,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1014980,1,3,2013-01-02,5530,668,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1013865,1,4,2013-01-03,4327,578,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1012750,1,5,2013-01-04,4486,619,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1011635,1,6,2013-01-05,4997,635,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5574,1115,1,2015-07-27,10712,608,1,1,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
4459,1115,2,2015-07-28,8093,500,1,1,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
3344,1115,3,2015-07-29,7661,473,1,1,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"
2229,1115,4,2015-07-30,8405,502,1,1,0,1,d,c,5350.0,NaN,NaN,1,22.0,2012.0,"Mar,Jun,Sept,Dec"


In [49]:
test['Open']=test['Open'].fillna(0)
test.isnull().sum()

Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                             0
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment                       0
CompetitionDistance             96
CompetitionOpenSinceMonth    15216
CompetitionOpenSinceYear     15216
Promo2                           0
Promo2SinceWeek              17232
Promo2SinceYear              17232
PromoInterval                17232
dtype: int64

In [50]:
# ---------------------------
# Dealing with Dates
# ---------------------------
for df in [train, test]:
    # Extract basic date features
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)

In [51]:
# ---------------------------
# Dealing with missing Values
# ---------------------------
for df in [train, test]:
    #  missing values (nulls in PromoInterval will be dealed with in encoding)
    df['Promo2SinceWeek']=df['Promo2SinceWeek'].fillna(0)
    df['Promo2SinceYear']=df['Promo2SinceYear'].fillna(0)

    df['CompetitionDistance']=df['CompetitionDistance'].fillna(train['CompetitionDistance'].median())
    df['CompetitionOpenSinceMonth']=df['CompetitionOpenSinceMonth'].fillna(0)
    df['CompetitionOpenSinceYear']=df['CompetitionOpenSinceYear'].fillna(0)
    
train.isnull().sum()

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth         0
CompetitionOpenSinceYear          0
Promo2                            0
Promo2SinceWeek                   0
Promo2SinceYear                   0
PromoInterval                508031
Year                              0
Month                             0
Day                               0
WeekOfYear                        0
dtype: int64

In [52]:
# ---------------------------
# One Hot Encoding for categorial features
# ---------------------------
categorical_features = ['StoreType', 'Assortment', 'StateHoliday','PromoInterval']
for feature in categorical_features:
    if feature in train.columns and feature in test.columns:
        train = pd.get_dummies(train, columns=[feature], prefix=[feature])
        test = pd.get_dummies(test, columns=[feature], prefix=[feature])
train, test = train.align(test, join='left', axis=1, fill_value=0)
train.isnull().sum()       

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
SchoolHoliday                     0
CompetitionDistance               0
CompetitionOpenSinceMonth         0
CompetitionOpenSinceYear          0
Promo2                            0
Promo2SinceWeek                   0
Promo2SinceYear                   0
Year                              0
Month                             0
Day                               0
WeekOfYear                        0
StoreType_a                       0
StoreType_b                       0
StoreType_c                       0
StoreType_d                       0
Assortment_a                      0
Assortment_b                      0
Assortment_c                      0
StateHoliday_0                    0
StateHoliday_a                    0
StateHoliday_b              

In [53]:
# ---------------------------
# Choosing needed Features
# ---------------------------
features = ['Store','DayOfWeek','Year', 'Month', 'Day','Open', 'Promo', 'SchoolHoliday',  # Keep Open in the feature set
            'CompetitionDistance', 'CompetitionOpenSinceMonth','CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek','Promo2SinceYear'] + \
           [col for col in train.columns if 'StoreType_' in col or 'Assortment_' in col or 'StateHoliday_' in col or 'PromoInterval_' in col]

target = 'Sales'
train[features]

,Store,DayOfWeek,Year,Month,Day,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,...,Assortment_a,Assortment_b,Assortment_c,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
1016095,1,2,2013,1,1,0,0,1,1270.0,9.0,...,True,False,False,False,True,False,False,False,False,False
1014980,1,3,2013,1,2,1,0,1,1270.0,9.0,...,True,False,False,True,False,False,False,False,False,False
1013865,1,4,2013,1,3,1,0,1,1270.0,9.0,...,True,False,False,True,False,False,False,False,False,False
1012750,1,5,2013,1,4,1,0,1,1270.0,9.0,...,True,False,False,True,False,False,False,False,False,False
1011635,1,6,2013,1,5,1,0,1,1270.0,9.0,...,True,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5574,1115,1,2015,7,27,1,1,1,5350.0,0.0,...,False,False,True,True,False,False,False,False,False,True
4459,1115,2,2015,7,28,1,1,1,5350.0,0.0,...,False,False,True,True,False,False,False,False,False,True
3344,1115,3,2015,7,29,1,1,1,5350.0,0.0,...,False,False,True,True,False,False,False,False,False,True
2229,1115,4,2015,7,30,1,1,1,5350.0,0.0,...,False,False,True,True,False,False,False,False,False,True


In [54]:
# ---------------------------
# Standrization Function
# ---------------------------
# encoded columns will be the same only float
scaler = StandardScaler()
train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])
target_scaler = StandardScaler()
train['Sales'] = target_scaler.fit_transform(train[['Sales']])
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,...,Assortment_a,Assortment_b,Assortment_c,StateHoliday_0,StateHoliday_a,StateHoliday_b,StateHoliday_c,"PromoInterval_Feb,May,Aug,Nov","PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec"
1016095,-1.73164,-1.000476,2013-01-01,-1.499723,0,-2.210440,-0.785400,2.144211,-0.538742,0.950877,...,0.944815,-0.090668,-0.929468,-5.635633,7.014824,-0.081366,-0.063616,-0.363286,-0.636252,-0.325521
1014980,-1.73164,-0.499823,2013-01-02,-0.063331,668,0.452399,-0.785400,2.144211,-0.538742,0.950877,...,0.944815,-0.090668,-0.929468,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,-0.325521
1013865,-1.73164,0.000831,2013-01-03,-0.375805,578,0.452399,-0.785400,2.144211,-0.538742,0.950877,...,0.944815,-0.090668,-0.929468,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,-0.325521
1012750,-1.73164,0.501484,2013-01-04,-0.334505,619,0.452399,-0.785400,2.144211,-0.538742,0.950877,...,0.944815,-0.090668,-0.929468,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,-0.325521
1011635,-1.73164,1.002138,2013-01-05,-0.201775,635,0.452399,-0.785400,2.144211,-0.538742,0.950877,...,0.944815,-0.090668,-0.929468,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,-0.325521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5574,1.72897,-1.501129,2015-07-27,1.282670,608,0.452399,1.273237,2.144211,-0.009347,-1.150188,...,-1.058408,-0.090668,1.075884,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,3.072002
4459,1.72897,-1.000476,2015-07-28,0.602397,500,0.452399,1.273237,2.144211,-0.009347,-1.150188,...,-1.058408,-0.090668,1.075884,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,3.072002
3344,1.72897,-0.499823,2015-07-29,0.490187,473,0.452399,1.273237,2.144211,-0.009347,-1.150188,...,-1.058408,-0.090668,1.075884,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,3.072002
2229,1.72897,0.000831,2015-07-30,0.683437,502,0.452399,1.273237,2.144211,-0.009347,-1.150188,...,-1.058408,-0.090668,1.075884,0.177442,-0.142555,-0.081366,-0.063616,-0.363286,-0.636252,3.072002


In [55]:
# ---------------------------
# Sequence Function
# ---------------------------
# *data: The input DataFrame Training or Testing
# *seq_length: The length of the sequence (number of time steps) to use for predicting the next values.
# *features: The list of column names to use as input feature.
# *target: The target column (optional only used in training)
# *device: The device (either 'cuda' for GPU or 'cpu') where the tensors should be moved for computation.
def create_sequences_torch(data, seq_length, features, target=None, device='cuda'):
    # Convert feature data to tensor and move to device(GPU).
    data_tensor = torch.tensor(data[features].values, dtype=torch.float32).to(device)
    # if target != None 
    if target:
        target_tensor = torch.tensor(data[target].values, dtype=torch.float32).to(device)
    # Initialize sequences
    # X is the training input,y is the target (sales) 
    X, y = [], []
    #This retrieves all unique store identifiers from the Store column.
    stores = data['Store'].unique()
    # Sequences counter for print
    total_sequences = 0
    #The loop ensures that the data for each store is handled separately
    for store in stores:
        #using boolean indexing to select the all feetaures for the same store in al dates
        store_indices = data['Store'] == store
        store_data = data_tensor[store_indices]
        if target:
            store_target = target_tensor[store_indices]
        # Grouping all features of the store in the past 7 days
        for i in range(len(store_data) - seq_length):
            X.append(store_data[i:i+seq_length])
            if target:
                y.append(store_target[i+seq_length])
            total_sequences += 1
            # Print progress for every 100000 sequences
            if total_sequences % 100000 == 0:
                print(f"Generated {total_sequences} sequences...")
    # Stack all sequences
    X = torch.stack(X).to(device)
    y = torch.stack(y).to(device) if target else None
    if target :
        return X, y
    else :
        return X

 #  use the past 7 days to predict the next day
sequence_length = 7 
# choose GPU if availbe
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
# Ensure the data is sorted by 'Store' and 'Date' before resetting the index
train = train.sort_values(by=['Store', 'Date']).reset_index(drop=True)




Using device: cuda


In [57]:
# ---------------------------
# Create Training sequence
# ---------------------------
X_train, y_train = create_sequences_torch(train, sequence_length, features, target, device=device)
# Convert tensors to NumPy arrays
X_train = X_train.cpu().numpy()
y_train = y_train.cpu().numpy()
# Save to .npy files
np.save('/kaggle/working/X_train.npy', X_train)
np.save('/kaggle/working/y_train.npy', y_train)

print("Sequences saved to .npy files!")


Sequences saved to .npy files!


In [58]:
# ---------------------------
# Define LSTM model
# ---------------------------
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, len(features))),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 50)             │        15,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,851 (61.92 KB)

 Trainable params: 15,851 (61.92 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
# ---------------------------
# Train  model
# ---------------------------
X_train, y_train = shuffle(X_train, y_train, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss',  # Monitor validation loss
                               patience=3,  # Number of epochs to wait for improvement
                               restore_best_weights=True,  # Restore best model weights after stopping
                               verbose=1)
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2,callbacks=[early_stopping])

Epoch 1/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 66s 3ms/step - loss: 0.4767 - val_loss: 0.3237
Epoch 2/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.3032 - val_loss: 0.2094
Epoch 3/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.2216 - val_loss: 0.1691
Epoch 4/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1844 - val_loss: 0.1396
Epoch 5/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1606 - val_loss: 0.1254
Epoch 6/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1442 - val_loss: 0.1128
Epoch 7/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1327 - val_loss: 0.1049
Epoch 8/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1236 - val_loss: 0.0974
Epoch 9/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1191 - val_loss: 0.0960
Epoch 10/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1156 - val_loss: 0.0981
Epoch 11/20
25236/25236 ━━━━━━━━━━━━━━━━━━━━ 61s 2ms/step - loss: 0.1136 - val_loss: 0.09

In [60]:
# ---------------------------
# Prepare test data sequences
# ---------------------------
X_test = create_sequences_torch(test, sequence_length, features)
# Convert to numpy
X_test = X_test.cpu().numpy()

In [62]:
# ---------------------------
# Predict Sales
# ---------------------------
# Predict sales
predictions = model.predict(X_test)
# Transform predictions back to original scale
predictions_original_scale = target_scaler.inverse_transform(predictions)
# ---------------------------
# Mapping Predictions with test data
# ---------------------------
# Create a sorted copy of the test data sorted by 'Store' and 'Date'
test_sorted_copy = test_sorted.copy().sort_values(by=['Store', 'Date']).reset_index(drop=True)
# Initialize a list to store the final predictions
final_predictions = []
# Initialize a pointer to keep track of predictions used
predictions_ptr = 0
# Get the unique stores
unique_stores = test_sorted_copy['Store'].unique()
# Iterate through each store to assign predictions
for store in unique_stores:
    # Extract data for the current store
    store_data = test_sorted_copy[test_sorted_copy['Store'] == store].sort_values('Date').reset_index(drop=True)
    n = len(store_data)
    
    # Number of actual predictions available for this store
    num_preds = n - sequence_length 
    # Extract predictions for this store
    store_preds = predictions_original_scale[predictions_ptr : predictions_ptr + num_preds]
    # Update the pointer
    predictions_ptr += num_preds
    # Calculate the mean of the store's predictions
    if len(store_preds) > 0:
        mean_value = np.mean(store_preds)
    else:
        # If no predictions are available, set mean_value to a default (e.g., overall mean or 0)
        mean_value = 0
    # Fill the first 'sequence_length' predictions with the mean
    # Ensure that we don't exceed the number of available entries
    missing_values = [mean_value] * min(sequence_length, n)
    # Flatten the predictions if they are multi-dimensional
    actual_predictions = store_preds.flatten().tolist()
    # Combine missing and actual predictions
    final_predictions.extend(missing_values + actual_predictions)
# ---------------------------
#  Validate the Number of Predictions
# ---------------------------
# Verify that the number of predictions matches the number of test samples
if len(final_predictions) != len(test_sorted_copy):
    raise ValueError(f"Length mismatch: {len(final_predictions)} predictions vs {len(test_sorted_copy)} test samples.")
# Assign the predictions to the sorted test data
test_sorted_copy['Sales'] = final_predictions
# ---------------------------
#  Create Submission DataFrame
# ---------------------------
# Extract all 'Id' values from the sorted test data (do NOT slice)
test_ids_sorted = test_sorted_copy['Id'].values
# Ensure that 'Sales' is a 1D array for the DataFrame
sales_values = np.array(final_predictions).flatten()
# Verify lengths match (for debugging)
print(f"Number of 'Id's: {len(test_ids_sorted)}")
print(f"Number of 'Sales' predictions: {len(sales_values)}")
if len(test_ids_sorted) != len(sales_values):
    raise ValueError(f"Length mismatch after processing: {len(test_ids_sorted)} 'Id's vs {len(sales_values)} 'Sales' predictions.")
# Create the submission DataFrame with 'Id' and 'Sales' columns
submission = pd.DataFrame({
    'Id': test_ids_sorted,  # All 'Id's from the sorted test data
    'Sales': sales_values  # All predictions
})
submission = submission.sort_values(by='Id').reset_index(drop=True)
# ---------------------------
# Final Verification
# ---------------------------
# Check for any missing values
missing_sales = submission['Sales'].isnull().sum()
if missing_sales > 0:
    print(f"There are {missing_sales} missing predictions. Filling them with 0.")
    submission['Sales'].fillna(0, inplace=True)
# Ensure no negative sales
negative_sales = (submission['Sales'] < 0).sum()
if negative_sales > 0:
    print(f"There are {negative_sales} negative sales values. Setting them to 0.")
    submission.loc[submission['Sales'] < 0, 'Sales'] = 0
# ---------------------------
#  Save the Submission File
# ---------------------------
# Save the final submission file (only 'Id' and 'Sales' columns)
submission.to_csv('submission.csv', index=False)

print("Submission file created successfully!")

1097/1097 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Number of 'Id's: 41088
Number of 'Sales' predictions: 41088
There are 2350 negative sales values. Setting them to 0.
Submission file created successfully!
